In [ ]:
plt.close('all')

In [ ]:
# Plot the histograms of spot depths and spot sizes across all images
spot_depths_all_flattened = [depth for sublist in spot_depths_all for depth in sublist]
spot_sizes_all_flattened = [size * 34.332 for sublist in spot_sizes_all for size in sublist]  # convert to nm^2

plt.figure(figsize=(10, 14))

# Top subplot: Histogram of spot depths
plt.subplot(2, 1, 1)
plt.hist(spot_depths_all_flattened, bins=28, color='green', alpha=0.7)
plt.title('Histogram of Spot Depths Across All Images')
plt.xlabel('Spot Depth (mean intensity)')
plt.ylabel('Frequency')

# Bottom subplot: Histogram of spot sizes
plt.subplot(2, 1, 2)
plt.hist(spot_sizes_all_flattened, bins=28, color='blue', alpha=0.7)
plt.title('Histogram of Spot Sizes Across 10 3 $\mu m \\times$ 3 $\mu m$ AFM Images of Silane-Treated Sample')
plt.xlabel('Spot Size ($nm^2$)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
all_data_dict['si_old_zoomed_02.0_00000_grey.png'].keys()

In [ ]:
folder_path = r"C:\Users\cobia\OneDrive - University of Cambridge\Python\afm_data\all_images_03_08_2025"
label_files = [os.path.join(folder_path, f"{os.path.splitext(filename)[0]}_categorised.csv") for filename in all_data_dict.keys()]

for label_file in label_files:
    print(os.path.exists(label_file))

## Old Patch Generator (with median spot size calculation)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Step 1: Prepare the dataset
patches = []
labels = []

def load_feature_points(csv_path):
    """Load feature points from CSV file."""
    df = pd.read_csv(csv_path)
    source_points = df[['x2', 'y2']].values  # Feature positions in image 1
    target_points = df[['x1', 'y1']].values  # Corresponding positions in image 2
    return source_points, target_points

for file, data in all_data_dict.items():
    if True: #file == 'fat_end_outside_L_03.0_00000.png':
        print('file: ', file)
        spots = data['spots']
        print('spots: ', len(spots))
        spot_sizes = data['spot_sizes']
        afm_img = data['afm_img']
        rbf_x = data['rbf_x']
        rbf_y = data['rbf_y']
        cl_img = data['cl_img']
        cl_raw = data['cl_raw']
        cl_centre = data['m_centre'].data
        cl_intensity = data['m_intensity'].data
        cl_fwhm = data['m_fwhm'].data

        source_points, target_points = load_feature_points(data['alignment_file'])

        h, w = afm_img.shape[:2]

        # Create a grid of pixel coordinates
        grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
        grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

        # Interpolate transformation using RBF (thin-plate spline kernel)
        rbf_x = RBFInterpolator(source_points, target_points[:, 0], kernel='thin_plate_spline')
        rbf_y = RBFInterpolator(source_points, target_points[:, 1], kernel='thin_plate_spline')

        # Compute the transformed coordinates
        warped_x = rbf_x(grid_points).reshape(h, w).astype(np.float32)
        warped_y = rbf_y(grid_points).reshape(h, w).astype(np.float32)
        
        warped_image_afm = cv2.remap(afm_img, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)
        
        # Classify spots into largest 50% and smallest 50%
        median_size = np.percentile(spot_sizes, 41) #np.median(spot_sizes)

        cl_spot_coords = []

        for spot, size in zip(spots, spot_sizes):
            # Get the AFM spot coordinates
            afm_coords = np.array(spot.centroid)
            
            # Map to CL image coordinates
            blank_array = np.zeros_like(afm_img)
            blank_array[int(afm_coords[0]), int(afm_coords[1])] = 255
            blank_array[int(afm_coords[0]), int(afm_coords[1]+1)] = 127
            blank_array[int(afm_coords[0]), int(afm_coords[1]-1)] = 127
            blank_array[int(afm_coords[0]+1), int(afm_coords[1])] = 127
            blank_array[int(afm_coords[0]-1), int(afm_coords[1])] = 127
            blank_array[int(afm_coords[0]+1), int(afm_coords[1]+1)] = 127
            blank_array[int(afm_coords[0]+1), int(afm_coords[1]-1)] = 127
            blank_array[int(afm_coords[0]-1), int(afm_coords[1]+1)] = 127
            blank_array[int(afm_coords[0]-1), int(afm_coords[1]-1)] = 127
            # Use RBF to get CL coordinates
            warped_image = cv2.remap(blank_array, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)
            # Find the maximum value in warped_image
            max_value = np.max(warped_image)
            # Find the coordinates of the maximum value
            max_coords = np.argwhere(warped_image == max_value)

            cl_x, cl_y = max_coords[0]  # Get the first coordinate pair
            print(f'AFM coordinates are {afm_coords[0]} and {afm_coords[1]}')
            print(f'CL coordinates are {cl_x} and {cl_y}')

            cl_spot_coords.append((cl_x, cl_y))

            # Extract a 15x15 patch around the spot
            half_size = 7  # Half of 15px

            print('cl_img size = ', cl_img.shape)

            if (cl_y - half_size >= 0 and cl_y + half_size < cl_img.shape[0] and
                cl_x - half_size >= 0 and cl_x + half_size < cl_img.shape[1]):
                patch_centre = cl_centre[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]
                patch_intensity = cl_intensity[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]
                patch_fwhm = cl_fwhm[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]
                patch_raw = cl_raw[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1, :]
                patch = np.stack((patch_centre,
                                  patch_intensity,
                                  patch_fwhm), axis=-1)
                # patch = patch_raw
                patches.append(patch)
                labels.append(1 if size > median_size else 0)
        
print('total number of patches = ', len(patches))
print('total number of labels = ', len(labels))

# Convert to numpy arrays
patches = np.array(patches)
labels = np.array(labels)

print(patches.shape)

## New Spot Generator (from label files)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import cv2
from scipy.interpolate import RBFInterpolator

def load_feature_points(csv_path):
    """Load feature points from CSV file."""
    df = pd.read_csv(csv_path)
    source_points = df[['x1', 'y1']].values  # Feature positions in image 1
    target_points = df[['x2', 'y2']].values  # Corresponding positions in image 2
    return source_points, target_points


folder_path = r"C:\Users\cobia\OneDrive - University of Cambridge\Python\afm_data\all_images_03_08_2025"

patches = []
labels = []

for file, data in all_data_dict.items():
    print('file:', file)

    # Load the label file
    label_file = os.path.join(
        folder_path, f"{os.path.splitext(file)[0]}_categorised.csv"
    )
    label_df = pd.read_csv(label_file)

    afm_img = data['afm_img']
    cl_img = data['cl_img']
    cl_centre = data['m_centre'].data
    cl_intensity = data['m_intensity'].data
    cl_fwhm = data['m_fwhm'].data
    cl_raw = data['cl_raw']

    source_points, target_points = load_feature_points(data['alignment_file'])
    h, w = afm_img.shape[:2]

    # Prepare coordinate transform
    grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
    grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])
    # rbf_x = RBFInterpolator(source_points, target_points[:, 0], kernel='thin_plate_spline')
    # rbf_y = RBFInterpolator(source_points, target_points[:, 1], kernel='thin_plate_spline')
    rbf_x = RBFInterpolator(target_points, source_points[:, 0], kernel='thin_plate_spline')
    rbf_y = RBFInterpolator(target_points, source_points[:, 1], kernel='thin_plate_spline')

    warped_x = rbf_x(grid_points).reshape(h, w).astype(np.float32)
    warped_y = rbf_y(grid_points).reshape(h, w).astype(np.float32)

    # Loop through labeled spots
    for _, row in label_df.iterrows():
        if row['category'] not in ['large', 'small']:
            continue  # skip unlabeled or irrelevant

        afm_x, afm_y = row['x'], 511-row['y']  # adjust y as per original plotting code

        # Create a small mask at AFM coords
        blank_array = np.zeros_like(afm_img)
        offsets = [(0, 0), (0, 1), (0, -1), (1, 0), (-1, 0),
                   (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dy, dx in offsets:
            yy = int(afm_y + dy)
            xx = int(afm_x + dx)
            if 0 <= yy < blank_array.shape[0] and 0 <= xx < blank_array.shape[1]:
                blank_array[yy, xx] = 255 if (dy, dx) == (0, 0) else 127

        # Warp to CL coords
        warped_image = cv2.remap(blank_array, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)
        max_coords = np.argwhere(warped_image == np.max(warped_image))
        cl_x, cl_y = max_coords[0]  # first max coordinate

        # Extract patch around CL spot
        half_size = 7
        if (cl_y - half_size >= 0 and cl_y + half_size < cl_img.shape[0] and
            cl_x - half_size >= 0 and cl_x + half_size < cl_img.shape[1]):
            patch_centre = cl_centre[cl_y - half_size:cl_y + half_size + 1,
                                     cl_x - half_size:cl_x + half_size + 1]
            patch_intensity = cl_intensity[cl_y - half_size:cl_y + half_size + 1,
                                           cl_x - half_size:cl_x + half_size + 1]
            patch_fwhm = cl_fwhm[cl_y - half_size:cl_y + half_size + 1,
                                 cl_x - half_size:cl_x + half_size + 1]
            patch = np.stack((patch_centre, patch_intensity, patch_fwhm), axis=-1)

            patches.append(patch)
            labels.append(1 if row['category'] == 'large' else 0)

print('Total number of patches:', len(patches))
print('Total number of labels:', len(labels))

# Convert to numpy arrays
patches = np.array(patches)
labels = np.array(labels)

print('Patches shape:', patches.shape)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os

def show_afm_cl_with_labels(afm_img, cl_img, label_file, rbf_x, rbf_y):
    """
    afm_img: 2D numpy array (AFM data)
    cl_img:  2D numpy array (CL data)
    label_file: path to the *_categorised.csv file
    rbf_x, rbf_y: fitted RBFInterpolators mapping AFM -> CL coordinates
    """

    # Load labels
    labels_df = pd.read_csv(label_file)

    # Extract relevant rows (large/small spots only)
    labels_df = labels_df[labels_df['category'].isin(['large', 'small'])].reset_index(drop=True)

    # Prepare figure
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # ---- AFM image ----
    axes[0].imshow(afm_img, cmap='afmhot')
    axes[0].set_title("AFM image (original coords)")
    axes[0].axis('off')

    for idx, row in labels_df.iterrows():
        afm_x, afm_y = row['x'], 511-row['y']
        # AFM image is usually (row = y, col = x), but labels may be in x,y
        axes[0].scatter(afm_x, afm_img.shape[0] - afm_y, c='cyan', s=20)
        axes[0].text(afm_x + 2, afm_img.shape[0] - afm_y, str(idx), color='white', fontsize=8)

    # ---- CL image ----
    axes[1].imshow(cl_img, cmap='viridis')
    axes[1].set_title("CL image (mapped coords)")
    axes[1].axis('off')

    for idx, row in labels_df.iterrows():
        afm_x, afm_y = row['x'], row['y']

        # Map coordinates: RBF expects col, row (x, y)
        mapped_x = rbf_x([[afm_x, afm_y]])[0]
        mapped_y = rbf_y([[afm_x, afm_y]])[0]

        # Scatter in CL image coords
        axes[1].scatter(mapped_x, mapped_y, c='cyan', s=20)
        axes[1].text(mapped_x + 2, mapped_y, str(idx), color='white', fontsize=8)

    plt.tight_layout()
    plt.show()

# Example usage:
for afm_img, cl_img, label_file, rbf_x, rbf_y in zip(afm_img_list, cl_img_list, label_file_list, rbf_x_list, rbf_y_list):
    show_afm_cl_with_labels(afm_img, cl_img, label_file, rbf_x, rbf_y) 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import os
import cv2

def interactive_cl_view(afm_img, cl_img, label_file, rbf_x, rbf_y, cl_intensity):
    """
    Interactive viewer for CL image patches.
    Click a CL spot to display its patch and draw bounding box.

    afm_img: 2D numpy array (AFM data)
    cl_img:  2D numpy array (CL data)
    label_file: path to *_categorised.csv file
    rbf_x, rbf_y: RBFInterpolators (CL→AFM fit, used AFM→CL mapping)
    cl_intensity: 2D numpy array of CL intensity channel
    """
    label_df = pd.read_csv(label_file)
    label_df = label_df[label_df['category'].isin(['large', 'small'])].reset_index(drop=True)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(afm_img, cmap='afmhot')
    axes[0].set_title("AFM Image")
    axes[0].axis('off')

    # Plot AFM points
    for idx, row in label_df.iterrows():
        afm_x, afm_y = row['x'], row['y']
        axes[0].scatter(afm_x, afm_y, c='cyan', s=20)
        axes[0].text(afm_x + 2, afm_y, str(idx), color='white', fontsize=8)

    axes[1].imshow(cl_img, cmap='viridis')
    axes[1].set_title("CL Image (click to view patch)")
    axes[1].axis('off')

    cl_points = []
    for idx, row in label_df.iterrows():
        afm_x, afm_y = row['x'], row['y']
        mapped_x = rbf_x([[afm_x, afm_y]])[0]
        mapped_y = rbf_y([[afm_x, afm_y]])[0]
        cl_points.append((mapped_x, mapped_y))
        axes[1].scatter(mapped_x, mapped_y, c='cyan', s=20)
        axes[1].text(mapped_x + 2, mapped_y, str(idx), color='white', fontsize=8)

    rect_artist = None

    def onclick(event):
        nonlocal rect_artist
        if event.inaxes != axes[1]:
            return

        # Find nearest spot
        click_x, click_y = event.xdata, event.ydata
        dists = [np.hypot(px - click_x, py - click_y) for px, py in cl_points]
        nearest_idx = int(np.argmin(dists))
        spot_x, spot_y = cl_points[nearest_idx]

        # Round to integer coords
        spot_x = int(round(spot_x))
        spot_y = int(round(spot_y))

        # Draw bounding box on CL image
        half_size = 7
        if rect_artist:
            rect_artist.remove()
        rect_artist = patches.Rectangle(
            (spot_x - half_size, spot_y - half_size),
            15, 15, linewidth=1.5, edgecolor='red', facecolor='none'
        )
        axes[1].add_patch(rect_artist)
        fig.canvas.draw_idle()

        # Extract patch
        if (spot_y - half_size >= 0 and spot_y + half_size < cl_intensity.shape[0] and
            spot_x - half_size >= 0 and spot_x + half_size < cl_intensity.shape[1]):
            patch = cl_intensity[spot_y - half_size:spot_y + half_size + 1,
                                 spot_x - half_size:spot_x + half_size + 1]

            # Show patch in new figure
            plt.figure(figsize=(3, 3))
            plt.imshow(patch, cmap='inferno')
            plt.title(f"Patch for spot {nearest_idx}")
            plt.axis('off')
            plt.show()

    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.tight_layout()
    plt.show()

# Example usage:
interactive_cl_view(
    afm_img,
    cl_img,
    label_file,
    rbf_x,
    rbf_y,
    cl_intensity=data['m_intensity'].data
)


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from scipy.interpolate import RBFInterpolator

patches = []
labels = []

afm_img_list, cl_img_list, label_file_list, rbf_x_list, rbf_y_list = [],[],[],[],[]

def load_feature_points(csv_path):
    """Load feature points from CSV file."""
    df = pd.read_csv(csv_path)
    source_points = df[['x2', 'y2']].values  # Feature positions in image 1
    target_points = df[['x1', 'y1']].values  # Corresponding positions in image 2
    return source_points, target_points

for file, data in all_data_dict.items():
    print('Processing file:', file)

    # Load label file
    label_file = os.path.join(
        folder_path, f"{os.path.splitext(file)[0]}_categorised.csv"
    )
    label_file_list.append(label_file)
    label_df = pd.read_csv(label_file)
    label_df = label_df[label_df['category'].isin(['large', 'small'])].reset_index(drop=True)

    # Extract images
    afm_img = data['afm_img']
    afm_img_list.append(afm_img)
    cl_img = data['cl_img']
    cl_centre = data['m_centre'].data
    cl_intensity = data['m_intensity'].data
    cl_img_list.append(cl_intensity)
    cl_fwhm = data['m_fwhm'].data
    cl_raw = data['cl_raw']

    # Load feature points (alignment)
    source_points, target_points = load_feature_points(data['alignment_file'])

    # Fit RBF in CL → AFM direction (to match your working setup)
    rbf_x = RBFInterpolator(target_points, source_points[:, 0], kernel='thin_plate_spline')
    rbf_y = RBFInterpolator(target_points, source_points[:, 1], kernel='thin_plate_spline')

    rbf_x_list.append(rbf_x)
    rbf_y_list.append(rbf_y)

    # Process each labelled spot
    for _, row in label_df.iterrows():
        afm_x, afm_y = row['x'], 511 - row['y']  # Y-flip for AFM coordinates

        # Map AFM coords to CL coords
        mapped_x = rbf_x([[afm_x, afm_y]])[0]
        mapped_y = rbf_y([[afm_x, afm_y]])[0]

        # Round to nearest int for indexing
        cl_x, cl_y = int(round(mapped_x)), int(round(mapped_y))

        half_size = 7  # Half of 15 px patch

        # Ensure patch is within bounds
        if (cl_y - half_size >= 0 and cl_y + half_size < cl_img.shape[0] and
            cl_x - half_size >= 0 and cl_x + half_size < cl_img.shape[1]):

            patch_centre = cl_centre[cl_y - half_size:cl_y + half_size + 1,
                                     cl_x - half_size:cl_x + half_size + 1]
            patch_intensity = cl_intensity[cl_y - half_size:cl_y + half_size + 1,
                                           cl_x - half_size:cl_x + half_size + 1]
            patch_fwhm = cl_fwhm[cl_y - half_size:cl_y + half_size + 1,
                                 cl_x - half_size:cl_x + half_size + 1]

            patch = np.stack((patch_centre, patch_intensity, patch_fwhm), axis=-1)
            patches.append(patch)

            # Label: large = 1, small = 0
            labels.append(1 if row['category'] == 'large' else 0)

print('Total patches:', len(patches))
print('Total labels:', len(labels))

patches = np.array(patches)
labels = np.array(labels)

print('Patch array shape:', patches.shape)
print('Label array shape:', labels.shape)



In [ ]:
plt.close('all')

In [ ]:
import matplotlib.pyplot as plt
import random

# Pick 10 random indices
num_to_show = min(20, len(patches))
indices = random.sample(range(len(patches)), num_to_show)

fig, axes = plt.subplots(4, 5, figsize=(9, 5))
axes = axes.flatten()

for ax, idx in zip(axes, indices):
    patch = patches[idx,:,:,1]
    label = 'Large' if labels[idx] == 1 else 'Small'
    
    # Display as RGB-like by normalizing per channel
    patch_display = patch.copy().astype(float)
    # for c in range(patch_display.shape[-1]):
    #     ch_min, ch_max = patch_display[..., c].min(), patch_display[..., c].max()
    #     if ch_max > ch_min:
    #         patch_display[..., c] = (patch_display[..., c] - ch_min) / (ch_max - ch_min)
    
    ax.imshow(patch_display, cmap='viridis')
    ax.set_title(f"{label} Spot")
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(11, 5))

# Plot AFM image with numbered spots
axes[0].imshow(afm_img, cmap='gray')
axes[0].set_title("AFM Image with Numbered Spots")
for i, spot in enumerate(spots):
    cy, cx = spot.centroid
    # axes[0].scatter(cx, cy, color='red', s=20)
    axes[0].text(cx, cy, str(i + 1), color='yellow', fontsize=8, ha='center', va='center')
axes[0].axis('off')

axes[1].imshow(blank_array, cmap='grey')
axes[1].set_title("Generated Image with 1 White Spot (Spot 48)")
axes[1].axis('off')

# Plot CL image with numbered spots
axes[2].imshow(cl_img, cmap='viridis')
axes[2].imshow(warped_image_afm, alpha=0.3, cmap='afmhot')
axes[2].set_title("CL Image with Numbered Spots")
for i, (cl_x, cl_y) in enumerate(cl_spot_coords):
    # axes[1].scatter(cl_y, cl_x, color='blue', s=20)
    axes[2].text(cl_y, cl_x, str(i + 1), color='white', fontsize=8, ha='center', va='center')
axes[2].set_xlim(0, 256)
axes[2].set_ylim(256, 0)
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(patches[0], cmap='viridis')
plt.show()

In [ ]:
# One-hot encode labels
labels = to_categorical(labels, num_classes=2)

In [ ]:
labels

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Flatten the patches for logistic regression input
X = patches.reshape(patches.shape[0], -1)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Step 2: Build the Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Step 3: Train the model
model.fit(X_train, y_train.argmax(axis=1))  # Use argmax to convert one-hot labels to class indices

# Step 4: Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Compute the confusion matrix
cm = confusion_matrix(y_test.argmax(axis=1), y_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Small', 'Large'])
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title('Logistic Regession - Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.tight_layout()
plt.show()


In [ ]:
# Normalize patches
# patches[:,:,:,0] = (patches[:,:,:,0] - patches[:,:,:,0].min()) / (patches[:,:,:,0].max() - patches[:,:,:,0].min())
# patches[:,:,:,1] = (patches[:,:,:,1] - patches[:,:,:,1].min()) / (patches[:,:,:,1].max() - patches[:,:,:,1].min())
# patches[:,:,:,2] = (patches[:,:,:,2] - patches[:,:,:,2].min()) / (patches[:,:,:,2].max() - patches[:,:,:,2].min())

patches = (patches - patches.min()) / (patches.max() - patches.min())

# Reshape patches for CNN input
patches = patches.reshape(-1, 15, 15, 3)



# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(patches, labels, test_size=0.2, random_state=42)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
## New Model Definition 14/07/2025
from tensorflow.keras.layers import Activation  # Import Activation layer

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=(15, 15, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))  # Adjusted strides to prevent shrinking too much

model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))  # Adjusted strides to prevent shrinking too much

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf

# Ensure CPU only
tf.config.set_visible_devices([], 'GPU')

def dice_loss(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - ((2. * intersection + smooth) /
                (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth))

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

# # Build the CNN model
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(15, 15, 3)),  # Updated input_shape to (15, 15, 3)
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 3: Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

model.fit_generator(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test), steps_per_epoch=len(X_train) // 32) 

# Step 4: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def add_layer(ax, x, y, width, height, label, color='skyblue'):
    rect = patches.Rectangle((x, y), width, height, linewidth=1.5, edgecolor='k', facecolor=color)
    ax.add_patch(rect)
    ax.text(x + width/2, y + height + 0.1, label, ha='center', fontsize=10)

def draw_model():
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.set_xlim(0, 12)
    ax.set_ylim(0, 3)
    ax.axis('off')

    layer_params = [
        (0.5, 1.0, 1.0, 1.0, 'Input\n15×15×3', 'lightgrey'),
        (2.0, 1.0, 1.2, 1.2, 'Conv2D\n32@3×3', 'skyblue'),
        (3.7, 1.0, 1.2, 1.2, 'MaxPool\n2×2', 'lightgreen'),
        (5.4, 1.0, 0.8, 1.2, 'Flatten', 'orange'),
        (6.7, 1.0, 1.4, 1.2, 'Dense\n64 units', 'salmon'),
        (8.6, 1.0, 1.2, 1.2, 'Output\n2-class\nSoftmax', 'plum')
    ]

    for x, y, w, h, label, color in layer_params:
        add_layer(ax, x, y, w, h, label, color)

    # Optional arrows
    for i in range(len(layer_params)-1):
        x_start = layer_params[i][0] + layer_params[i][2]
        x_end = layer_params[i+1][0]
        y = 1.6
        ax.annotate('', xy=(x_end, y), xytext=(x_start, y),
                    arrowprops=dict(arrowstyle='->', lw=1.5))

    plt.title("CNN Architecture (Keras Sequential Model)", fontsize=12)
    plt.tight_layout()
    plt.show()

draw_model()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_layer(ax, x, y, width, height, depth, color='skyblue', label=None, max_depth_drawn=10):
    """
    Draw a stack of rectangles to represent a layer.
    Only up to `max_depth_drawn` layers are drawn for clarity.
    """
    depth_step = 0.2  # depth offset for layering
    num_to_draw = min(depth, max_depth_drawn)
    for i in range(num_to_draw):
        offset = i * depth_step
        rect = patches.Rectangle((x - offset, y - offset), width, height, linewidth=1, edgecolor='black', facecolor=color, alpha=0.6)
        ax.add_patch(rect)
    
    if label:
        ax.text(x + width/2, y + height + 1.5, label, ha='center', va='bottom', fontsize=8)

# Create figure and axes
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(0, 50)
ax.set_ylim(0, 20)
ax.axis('off')

# Define and draw each layer
layers = [
    {"x": 2, "y": 5, "w": 4, "h": 4, "d": 3,  "label": "Input\n15x15x3"},
    {"x": 8, "y": 5, "w": 4, "h": 4, "d": 32, "label": "Conv2D\n13x13x32"},
    {"x": 14, "y": 5, "w": 3, "h": 3, "d": 32, "label": "MaxPool\n6x6x32"},
    {"x": 20, "y": 5, "w": 2.5, "h": 2.5, "d": 1, "label": "Flatten\n1152"},
    {"x": 26, "y": 5, "w": 2, "h": 2, "d": 1, "label": "Dense\n64"},
    {"x": 32, "y": 5, "w": 1.5, "h": 1.5, "d": 1, "label": "Output\n2 (Softmax)"}
]

for layer in layers:
    draw_layer(ax, layer["x"], layer["y"], layer["w"], layer["h"], layer["d"], label=layer["label"])

plt.title("LeNet-style CNN Architecture", fontsize=12)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_layer(ax, x, y, w, h, depth, color, label, offset=0.2):
    for i in range(depth):
        rect = patches.Rectangle((x + i * offset, y - i * offset), w, h, 
                                 linewidth=1, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
    ax.text(x + depth * offset + 0.5, y + h / 2, label, fontsize=10, va='center')

def draw_kernel_connection(ax, x1, y1, w1, h1, x2, y2, w2, h2, kernel_size):
    # Kernel outline on input layer
    kx, ky = x1 + w1/2 - kernel_size/2, y1 + h1/2 - kernel_size/2
    ax.add_patch(patches.Rectangle((kx, ky), kernel_size, kernel_size, 
                                   linewidth=2, edgecolor='black', facecolor='none'))
    # Arrow to next layer
    ax.annotate('', xy=(x2, y2 + h2/2), xytext=(kx + kernel_size, ky + kernel_size/2),
                arrowprops=dict(arrowstyle='->', lw=1.5))

fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(0, 20)
ax.set_ylim(0, 10)
ax.axis('off')

# Input layer: 15x15x3
draw_layer(ax, 1, 4, 3, 3, 3, 'blue', 'Input\n15×15×3')

# Conv layer: 32@13x13
draw_layer(ax, 5, 4, 2.6, 2.6, 5, 'green', 'Conv2D\n32×13×13')
draw_kernel_connection(ax, 1, 4, 3, 3, 5, 4, 2.6, 2.6, kernel_size=0.6)

# MaxPooling layer: 32@6x6
draw_layer(ax, 9, 4, 1.2, 1.2, 5, 'purple', 'MaxPool\n32×6×6')

# Flatten: vector of 1152
draw_layer(ax, 12, 4.5, 0.6, 1, 1, 'orange', 'Flatten\n1152')

# Dense 64
draw_layer(ax, 14, 4.5, 0.6, 1, 3, 'red', 'Dense\n64')

# Output layer: 2 classes
draw_layer(ax, 16, 4.5, 0.6, 1, 2, 'black', 'Output\n2')

plt.tight_layout()
plt.show()


## AFM only Dataset Creation


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Step 1: Prepare the dataset
patches = []
labels = []

for file, data in all_data_dict.items():
    if True:
        print('file: ', file)
        spots = data['spots']
        print('spots: ', len(spots))
        spot_sizes = data['spot_sizes']
        afm_img = data['afm_img']
        # rbf_x = data['rbf_x']
        # rbf_y = data['rbf_y']
        # cl_img = data['cl_img']
        # cl_raw = data['cl_raw']
        # cl_centre = data['m_centre'].data
        # cl_intensity = data['m_intensity'].data
        # cl_fwhm = data['m_fwhm'].data

        # source_points, target_points = load_feature_points(data['alignment_file'])

        # h, w = afm_img.shape[:2]

        # # Create a grid of pixel coordinates
        # grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
        # grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

        # # Interpolate transformation using RBF (thin-plate spline kernel)
        # rbf_x = RBFInterpolator(source_points, target_points[:, 0], kernel='thin_plate_spline')
        # rbf_y = RBFInterpolator(source_points, target_points[:, 1], kernel='thin_plate_spline')

        # # Compute the transformed coordinates
        # warped_x = rbf_x(grid_points).reshape(h, w).astype(np.float32)
        # warped_y = rbf_y(grid_points).reshape(h, w).astype(np.float32)
        
        # warped_image_afm = cv2.remap(afm_img, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)
        
        # Classify spots into largest 50% and smallest 50%
        median_size = np.percentile(spot_sizes, 41) #np.median(spot_sizes)

        # cl_spot_coords = []

        for spot, size in zip(spots, spot_sizes):
            # Get the AFM spot coordinates
            afm_coords = np.array(spot.centroid)
            
            # # Map to CL image coordinates
            # blank_array = np.zeros_like(afm_img)
            # blank_array[int(afm_coords[0]), int(afm_coords[1])] = 255
            # blank_array[int(afm_coords[0]), int(afm_coords[1]+1)] = 127
            # blank_array[int(afm_coords[0]), int(afm_coords[1]-1)] = 127
            # blank_array[int(afm_coords[0]+1), int(afm_coords[1])] = 127
            # blank_array[int(afm_coords[0]-1), int(afm_coords[1])] = 127
            # blank_array[int(afm_coords[0]+1), int(afm_coords[1]+1)] = 127
            # blank_array[int(afm_coords[0]+1), int(afm_coords[1]-1)] = 127
            # blank_array[int(afm_coords[0]-1), int(afm_coords[1]+1)] = 127
            # blank_array[int(afm_coords[0]-1), int(afm_coords[1]-1)] = 127
            # # Use RBF to get CL coordinates
            # warped_image = cv2.remap(blank_array, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)
            # # Find the maximum value in warped_image
            # max_value = np.max(warped_image)
            # # Find the coordinates of the maximum value
            # max_coords = np.argwhere(warped_image == max_value)

            # cl_x, cl_y = max_coords[0]  # Get the first coordinate pair
            print(f'AFM coordinates are {afm_coords[0]} and {afm_coords[1]}')

            # AFM only: set cl_x and cl_y to afm_coords
            cl_x, cl_y = int(afm_coords[1]), int(afm_coords[0])  # Note: (x, y) -> (col, row)

            # print(f'CL coordinates are {cl_x} and {cl_y}')

            # cl_spot_coords.append((cl_x, cl_y))

            # Extract a 15x15 patch around the spot
            half_size = 7  # Half of 15px

            print('cl_img size = ', cl_img.shape)

            # if (cl_y - half_size >= 0 and cl_y + half_size < cl_img.shape[0] and
            #     cl_x - half_size >= 0 and cl_x + half_size < cl_img.shape[1]):
            #     patch_centre = cl_centre[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]
            #     patch_intensity = cl_intensity[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]
            #     patch_fwhm = cl_fwhm[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]
            #     patch_raw = cl_raw[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1, :]
            #     patch = np.stack((patch_centre,
            #                       patch_intensity,
            #                       patch_fwhm), axis=-1)
            #     # patch = patch_raw

            if (cl_y - half_size >= 0 and cl_y + half_size < cl_img.shape[0] and
                cl_x - half_size >= 0 and cl_x + half_size < cl_img.shape[1]):
                patch = afm_img[cl_y - half_size:cl_y + half_size + 1, cl_x - half_size:cl_x + half_size + 1]


                patches.append(patch)
                labels.append(1 if size > median_size else 0)
        
print('total number of patches = ', len(patches))
print('total number of labels = ', len(labels))

# Convert to numpy arrays
patches = np.array(patches)
labels = np.array(labels)



In [ ]:
import pandas as pd
import os

# Define the directory containing the CSV files
csv_directory = r"C:\Users\cobia\OneDrive - University of Cambridge\Python\afm_data - Copy\cobi\labels"

# List all CSV files in the directory
csv_files = [os.path.join(csv_directory, file) for file in os.listdir(csv_directory) if file.endswith('.csv')]

# Load all CSV files into a single pandas DataFrame, adding the filename as a column
dataframes = [pd.read_csv(file).assign(filename=os.path.basename(file)) for file in csv_files]
combined_dataframe = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
combined_dataframe

In [ ]:
afm_files

In [ ]:
import matplotlib.pyplot as plt

# Create a box and whisker plot
plt.figure(figsize=(8, 6))
combined_dataframe.boxplot(column='size', by='category', grid=False)

# Add labels and title
plt.xlabel('Category')
plt.ylabel('Size')
plt.title('Box and Whisker Plot: Size by Category')
plt.suptitle('')  # Remove the default title added by pandas

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the number of images to display
num_images = 50

# Create a figure with subplots
fig, axes = plt.subplots(int(num_images / 5) + 1, 5, figsize=(11, 30))

# Loop through the first 10 patches and display them
for i in range(num_images):
    row, col = divmod(i, 5)  # Calculate row and column indices
    axes[row, col].imshow(patches[i+1], cmap='gray')
    axes[row, col].axis('off')
    label = "Large" if labels[i+1][1] == 1 else "Small"
    axes[row, col].set_title(f"Patch {i+1}\n{label}")

plt.tight_layout()
plt.show()

### Tain CNN on AFM data

In [ ]:
# One-hot encode labels
labels = to_categorical(labels, num_classes=2)

In [ ]:
patches.shape

In [ ]:
patches = patches.reshape(-1, 15, 15, 1)
X_train, X_test, y_train, y_test = train_test_split(patches, labels, test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
## New Model Definition 14/07/2025
from tensorflow.keras.layers import Activation  # Import Activation layer

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=(15, 15, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))  # Adjusted strides to prevent shrinking too much

model.add(Conv2D(filters=96, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))  # Adjusted strides to prevent shrinking too much

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2, activation="softmax"))

In [ ]:
# patches = (patches - patches.min()) / (patches.max() - patches.min())

# # Reshape patches for CNN input and add a channel dimension
# patches = patches.reshape(-1, 15, 15, 1)  # Add channel dimension (e.g., grayscale images)



# # Split into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(patches, labels, test_size=0.2, random_state=42)

# Step 2: Build the CNN model
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(15, 15, 1)),  # Updated input_shape to (15, 15, 1)
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(2, activation='softmax')
# ])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test), steps_per_epoch=len(X_train) // 32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from importlib.metadata import requires
print(requires("pySPM"))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true_classes = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to class labels

# Compute the confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Display the confusion matrix with increased font size
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Small', 'Large'])
disp.plot(cmap='Blues')# , xticks_rotation=45, text_kw={'fontsize': 22})  # Increase font size for text
plt.title('3D CNN Confusion Matrix')#, fontsize=20)  # Increase title font size
plt.xlabel('Predicted Labels')#, fontsize=18)  # Increase x-axis label font size
plt.ylabel('True Labels')#, fontsize=18)  # Increase y-axis label font size

# Increase font size of axis tick mark labels
#plt.xticks(fontsize=14)
#plt.yticks(fontsize=14)

# Adjust legend font size
#plt.legend(fontsize=14)
plt.tight_layout()
plt.show()
# plt.savefig('confusion_matrix_afm.png', dpi=600, transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
im = ax.imshow(patches[0, :, :, 0], cmap='RdBu')
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Energy (eV) vs mean', fontsize=12)
ax.set_title('Example Patch Visualization', fontsize=14)
plt.show()

In [ ]:
patches[0].shape

### Make a slider to adjust parameters

In [ ]:
file = afm_files[0]  # Replace with your image file path

# Define the range of block sizes and offsets
block_sizes = [15, 31, 51, 71, 91]
offsets = [2000, 3000, 4000, 5000]

# Create a figure to display the results
fig, axes = plt.subplots(len(block_sizes), len(offsets), figsize=(11, 11))


image = io.imread(file, as_gray=True)
# Fit a plane to the image
x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
A = np.c_[x.ravel(), y.ravel(), np.ones_like(x.ravel())]
coeff, _, _, _ = np.linalg.lstsq(A, image.ravel(), rcond=None)
plane = (coeff[0] * x + coeff[1] * y + coeff[2]).reshape(image.shape)

# Set any pixel with a value higher than the plane to the level of the plane
filtered_image = np.minimum(image, plane)


# Loop through each combination of block size and offset
for i, block_size in enumerate(block_sizes):
    for j, offset in enumerate(offsets):
        # Perform local thresholding
        threshold_image = filters.threshold_local(filtered_image, block_size=block_size, offset=offset)
        dark_spots_image = filtered_image < threshold_image

        # Label the dark spots
        labeled_spots, num_spots = ndi.label(dark_spots_image)

        # Find the coordinates and sizes of the spots
        spots = measure.regionprops(labeled_spots)
        spot_sizes = [spot.area for spot in spots if 2 <= spot.area <= 100]

        axes[i,j].imshow(image, cmap='afmhot')
        for spot in spots:
            if 2 <= spot.area <= 100:
                cy, cx = spot.centroid
                contour = measure.find_contours(labeled_spots == spot.label, 0.5)
                for n, contour in enumerate(contour):
                    axes[i,j].plot(contour[:, 1], contour[:, 0], linewidth=1, color='blue')
        
        # Display the result
        
        axes[i, j].set_title(f'block_size={block_size}, offset={offset}')
        axes[i, j].axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


## Tkinter Game

In [ ]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk


# Define image
image1 = all_data_dict['fat_end_outside_L_03.0_00000.png']['afm_img']
spots_image1 = all_data_dict['fat_end_outside_L_03.0_00000.png']['spots']

# Initialize a DataFrame to store categorized spots
spot_data = pd.DataFrame(columns=['x', 'y', 'size', 'category'])

# CSV file path to save categorized spots
csv_file_path = 'categorized_spots.csv'

# Stack to keep track of categorized spots for undo functionality
undo_stack = []

# Counter to track the current spot index
current_spot_index = 0
total_spots = len([spot for spot in spots_image1 if 5 <= spot.area <= 100])

# Function to update the counter label
def update_counter_label():
    counter_label.config(text=f"Spot {current_spot_index + 1} of {total_spots}")

# Function to handle spot categorization
def categorize_spot(category, spot, canvas, spots_iter, root):
    global spot_data, undo_stack, current_spot_index, first_spot
    if category == "not_a_spot":
        print(f"Spot at ({spot.centroid[1]:.1f}, {spot.centroid[0]:.1f}) labeled as 'Not a Spot'.")
        # Push the categorized spot onto the undo stack
        undo_stack.append((spot, category))
    elif category == "combination":
        print(f"Spot at ({spot.centroid[1]:.1f}, {spot.centroid[0]:.1f}) labeled as 'Combination of Multiple Spots'.")

        # Temporarily commenting out the re-thresholding and user verification part

        # # Re-threshold the region around the spot
        # minr, minc, maxr, maxc = spot.bbox
        # sub_image = image1[minr:maxr, minc:maxc]
        # sub_threshold = filters.threshold_local(sub_image, block_size=11, offset=2048)
        # sub_dark_spots = sub_image < sub_threshold
        # sub_labeled_spots, _ = ndi.label(sub_dark_spots)
        # sub_spots = measure.regionprops(sub_labeled_spots)
        
        # # Display the re-thresholded region for user verification
        # sub_image_pil = Image.fromarray(sub_image)
        # sub_image_tk = ImageTk.PhotoImage(sub_image_pil)
        # sub_window = tk.Toplevel(root)
        # sub_window.title("Re-thresholded Spot Region")
        # sub_canvas = tk.Canvas(sub_window, width=sub_image.shape[1], height=sub_image.shape[0], bg="white")
        # sub_canvas.pack()
        # sub_canvas.create_image(0, 0, anchor=tk.NW, image=sub_image_tk)
        # sub_window.mainloop()
        # print("Please verify if the separation of spots was successful.")

        spot_data = pd.concat([spot_data, pd.DataFrame({
            'x': [spot.centroid[1]],
            'y': [spot.centroid[0]],
            'size': [spot.area],
            'category': [category]
        })], ignore_index=True)
        # Push the categorized spot onto the undo stack
        undo_stack.append((spot, category))

    else:
        # Add the spot to the DataFrame
        spot_data = pd.concat([spot_data, pd.DataFrame({
            'x': [spot.centroid[1]],
            'y': [spot.centroid[0]],
            'size': [spot.area],
            'category': [category]
        })], ignore_index=True)
        # Push the categorized spot onto the undo stack
        undo_stack.append((spot, category))
        print(f"Spot at ({spot.centroid[1]:.1f}, {spot.centroid[0]:.1f}) labeled as '{category}'.")

    # Save to CSV file
    spot_data.to_csv(csv_file_path, index=False)

    # Move to the next spot
    try:
        next_spot = next(spots_iter)
        first_spot = next_spot
        current_spot_index += 1
        update_counter_label()
        highlight_spot(next_spot, canvas)
    except StopIteration:
        print("All spots have been categorized.")
        root.destroy()

# Function to undo the last categorization
def undo_last_action(canvas, spots_iter):
    global spot_data, undo_stack, current_spot_index
    if undo_stack:
        last_spot, last_category = undo_stack.pop()
        # Remove the last categorized spot from the DataFrame
        spot_data = spot_data[~((spot_data['x'] == last_spot.centroid[1]) & 
                                (spot_data['y'] == last_spot.centroid[0]) & 
                                (spot_data['category'] == last_category))]
        print(f"Undo: Removed spot at ({last_spot.centroid[1]:.1f}, {last_spot.centroid[0]:.1f}) labeled as '{last_category}'.")
        # Highlight the undone spot again
        highlight_spot(last_spot, canvas)
        # Re-add the undone spot to the iterator
        print("New List: " + str([last_spot] + list(spots_iter)))
        spots_iter = iter([last_spot] + list(spots_iter))
        current_spot_index -= 1
        update_counter_label()
    else:
        print("Nothing to undo.")

# Function to highlight a spot on the canvas
def highlight_spot(spot, canvas):
    canvas.delete("highlight")
    x, y = spot.centroid[1], spot.centroid[0]
    canvas.create_oval(x-8, y-8, x+8, y+8, outline="yellow", width=2, tags="highlight")
    canvas.update()



# Create the main application window
root = tk.Tk()
root.title("Spot Categorization")

# Create a canvas to display the image
canvas = tk.Canvas(root, width=image1.shape[1], height=image1.shape[0], bg="white")
canvas.pack()

# Convert the image to a format suitable for tkinter
image1_pil = Image.fromarray(image1)
image1_tk = ImageTk.PhotoImage(image1_pil)
canvas.create_image(0, 0, anchor=tk.NW, image=image1_tk)

# Create a frame for buttons
button_frame = tk.Frame(root)
button_frame.pack(side=tk.BOTTOM, fill=tk.X)

# Create an iterator for the spots
spots_iter = iter([spot for spot in spots_image1 if 5 <= spot.area <= 100])

# Highlight the first spot
try:
    global first_spot
    first_spot = next(spots_iter)
    highlight_spot(first_spot, canvas)
except StopIteration:
    print("No spots to categorize.")
    root.destroy()

# Create a counter label
counter_label = tk.Label(root, text=f"Spot 1 of {total_spots}", font=("Arial", 12))
counter_label.pack()

# Create buttons for categorization
categories = [('Large', 'large'),
              ('Small', 'small'),
              ('Not a Spot', 'not_a_spot'),
              ('Combination', 'combination')]
for text, category in categories:
    button = ttk.Button(button_frame, text=text, command=lambda c=category: categorize_spot(c, first_spot, canvas, spots_iter, root))
    button.pack(side=tk.LEFT, padx=5, pady=5)
    button.pack(side=tk.LEFT, padx=5, pady=5)

# Add an Undo button
undo_button = ttk.Button(button_frame, text="Undo", command=lambda: undo_last_action(canvas, spots_iter))
undo_button.pack(side=tk.LEFT, padx=5, pady=5)

root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk


# Define image
image1 = all_data_dict['fat_end_outside_L_03.0_00000.png']['afm_img']
spots_image1 = all_data_dict['fat_end_outside_L_03.0_00000.png']['spots']

# Initialise a DataFrame to store categorised spots
spot_data = pd.DataFrame(columns=['x', 'y', 'size', 'category'])

# CSV file path to save categorised spots
csv_file_path = 'categorised_spots.csv'

# Stack to keep track of categorised spots for undo functionality
undo_stack = []

# Counter to track the current spot index
current_spot_index = 0
total_spots = len([spot for spot in spots_image1 if 5 <= spot.area <= 100])

# Function to update the counter label
def update_counter_label(manual=False):
    if manual:
        counter_label.config(text=f"Manual Categorisation. Click any undetected spots.")
    else:
        counter_label.config(text=f"Spot {current_spot_index + 1} of {total_spots}")

# Function to handle user clicks for undetected spots
def on_canvas_click(event):
    global spot_data, undo_stack
    x, y = event.x, event.y
    print(f"User clicked at ({x}, {y}).")

    # Create a pop-up window for categorisation
    popup = tk.Toplevel(root)
    popup.title("Categorise Spot")
    popup.geometry("300x150")

    label = tk.Label(popup, text=f"Categorise the spot at ({x}, {y}):", font=("Arial", 12))
    label.pack(pady=10)

    def categorise_new_spot(category):
        global spot_data, undo_stack
        print(f"Spot at ({x}, {y}) labeled as '{category}'.")
        # Add the new spot to the DataFrame
        spot_data = pd.concat([spot_data, pd.DataFrame({
            'x': [x],
            'y': [y],
            'size': [None],  # Size is unknown for manually added spots
            'category': [category]
        })], ignore_index=True)
        # Push the categorised spot onto the undo stack
        undo_stack.append(({'x': x, 'y': y}, category))
        # Save to CSV file
        spot_data.to_csv(csv_file_path, index=False)
        popup.destroy()

    # Buttons for categorisation
    large_button = ttk.Button(popup, text="Large", command=lambda: categorise_new_spot("large"))
    large_button.pack(side=tk.LEFT, padx=20, pady=20)

    small_button = ttk.Button(popup, text="Small", command=lambda: categorise_new_spot("small"))
    small_button.pack(side=tk.RIGHT, padx=20, pady=20)

# Function to highlight all spots
def highlight_all_spots():
    canvas.delete("highlight")
    for spot in spots_image1:
        x, y = spot.centroid[1], spot.centroid[0]
        canvas.create_oval(x-8, y-8, x+8, y+8, outline="yellow", width=2, tags="highlight")
    canvas.update()

# Function to finish the manual categorisation phase
def finish_manual_categorisation():
    print("Manual categorisation finished.")
    root.destroy()

# After all spots are categorised
def start_manual_categorisation():
    print("All spots have been categorised. Highlighting all spots.")
    update_counter_label(manual=True)
    highlight_all_spots()
    canvas.bind("<Button-1>", on_canvas_click)

    # Add a Finish button
    finish_button = ttk.Button(button_frame, text="Finish", command=finish_manual_categorisation)
    finish_button.pack(side=tk.LEFT, padx=5, pady=5)

# Modify the categorise_spot function to call start_manual_categorisation
def categorise_spot(category, spot, canvas, root):
    global spot_data, undo_stack, current_spot_index, first_spot, spots_iter
    if category == "skip":
        print('Skipping categorisation function for undo')
    elif category == "not_a_spot":
        print(f"Spot at ({spot.centroid[1]:.1f}, {spot.centroid[0]:.1f}) labeled as 'Not a Spot'.")
        undo_stack.append((spot, category))
    elif category == "combination":
        print(f"Spot at ({spot.centroid[1]:.1f}, {spot.centroid[0]:.1f}) labeled as 'Combination of Multiple Spots'.")
        spot_data = pd.concat([spot_data, pd.DataFrame({
            'x': [spot.centroid[1]],
            'y': [spot.centroid[0]],
            'size': [spot.area],
            'category': [category]
        })], ignore_index=True)
        undo_stack.append((spot, category))
    else:
        spot_data = pd.concat([spot_data, pd.DataFrame({
            'x': [spot.centroid[1]],
            'y': [spot.centroid[0]],
            'size': [spot.area],
            'category': [category]
        })], ignore_index=True)
        undo_stack.append((spot, category))
        print(f"Spot at ({spot.centroid[1]:.1f}, {spot.centroid[0]:.1f}) labeled as '{category}'.")

    spot_data.to_csv(csv_file_path, index=False)

    try:
        next_spot = next(spots_iter)
        first_spot = next_spot
        current_spot_index += 1
        update_counter_label()
        highlight_spot(next_spot, canvas)
    except StopIteration:
        start_manual_categorisation()

# Function to undo the last categorisation
def undo_last_action(canvas):
    global spot_data, undo_stack, current_spot_index, spots_iter
    if undo_stack:
        last_spot, last_category = undo_stack.pop()
        # Remove the last categorised spot from the DataFrame
        spot_data = spot_data[~((spot_data['x'] == last_spot.centroid[1]) & 
                                (spot_data['y'] == last_spot.centroid[0]) & 
                                (spot_data['category'] == last_category))]
        print(f"Undo: Removed spot at ({last_spot.centroid[1]:.1f}, {last_spot.centroid[0]:.1f}) labeled as '{last_category}'.")
        # Highlight the undone spot again
        highlight_spot(last_spot, canvas)
        # Re-add the undone spot to the iterator
        new_list = [last_spot] + list(spots_iter)
        print("New List: " + str(new_list))
        spots_iter = iter(new_list)
        current_spot_index -= 1
        update_counter_label()
        categorise_spot("skip", last_spot, canvas, root)
    else:
        print("Nothing to undo.")        


# Function to highlight a spot on the canvas
def highlight_spot(spot, canvas):
    canvas.delete("highlight")
    x, y = spot.centroid[1], spot.centroid[0]
    canvas.create_oval(x-8, y-8, x+8, y+8, outline="yellow", width=2, tags="highlight")
    canvas.update()



# Create the main application window
root = tk.Tk()
root.title("Spot Categorisation")

# Create a canvas to display the image
canvas = tk.Canvas(root, width=image1.shape[1], height=image1.shape[0], bg="white")
canvas.pack()

# Convert the image to a format suitable for tkinter
image1_pil = Image.fromarray(image1)
image1_tk = ImageTk.PhotoImage(image1_pil)
canvas.create_image(0, 0, anchor=tk.NW, image=image1_tk)

# Create a frame for buttons
button_frame = tk.Frame(root)
button_frame.pack(side=tk.BOTTOM, fill=tk.X)

# Create an iterator for the spots
spots_iter = iter([spot for spot in spots_image1 if 5 <= spot.area <= 100])

# Highlight the first spot
try:
    global first_spot
    first_spot = next(spots_iter)
    highlight_spot(first_spot, canvas)
except StopIteration:
    print("No spots to categorise.")
    root.destroy()

# Create a counter label
counter_label = tk.Label(root, text=f"Spot 1 of {total_spots}", font=("Arial", 12))
counter_label.pack()

# Create buttons for categorisation
categories = [('Large', 'large'),
              ('Small', 'small'),
              ('Not a Spot', 'not_a_spot'),
              ('Combination', 'combination')]
for text, category in categories:
    button = ttk.Button(button_frame, text=text, command=lambda c=category: categorise_spot(c, first_spot, canvas, root))
    button.pack(side=tk.LEFT, padx=5, pady=5)
    button.pack(side=tk.LEFT, padx=5, pady=5)

# Add an Undo button
undo_button = ttk.Button(button_frame, text="Undo", command=lambda: undo_last_action(canvas))
undo_button.pack(side=tk.LEFT, padx=5, pady=5)

root.mainloop()


## Gaussian Fit Figure Creation for FY Report

In [ ]:
cl_filename = all_data_dict['fat_end_outside_L_03.0_00000.png']['cl_file']
cl_sem = hs.load(cl_filename, signal_type='CL_SEM')
# Display the CL SEM image 
cl_sem.plot(image=True, cmap='viridis', title='CL SEM Image', figsize=(10, 10))

In [ ]:
cl_sem.T.plot()

In [ ]:
cl_sem_eV = cl_sem.to_eV(inplace=False, jacobian=False)

In [ ]:
cl_sem_eV.plot(image=True, cmap='viridis', title='CL SEM Image in eV', figsize=(10, 10))

In [ ]:
m = cl_sem_eV.create_model()
bkg = hs.model.components1D.Offset()
g1 = hs.model.components1D.GaussianHF()
m.extend([g1, bkg])
cl_sem_eV.axes_manager.indices = (17,17)
g1.centre.value = 3.4        # Gaussian centre
g1.fwhm.value = 0.1      # Gaussian width
g1.height.value = 100      # Gaussian height
bkg.offset.value = 0.1   # background offset
g1.centre.bmax = g1.centre.value + 0.2
g1.centre.bmin = g1.centre.value - 0.2
g1.fwhm.bmin = 0.01
m.fit()
m.assign_current_values_to_all()
m.plot()

# Figure for Poster bkg

In [ ]:
cl_filename = r"C:\Users\cobia\OneDrive - University of Cambridge\CL\COBI-20250501\HYP-SI-FAT-END-OUTSIDE-07+6UM\HYPCard_corrected.hspy"

im = hs.load(cl_filename, signal_type='CL_SEM').T

im.plot()
roi1 = hs.roi.SpanROI(left=350, right=380) #sets a digitalbandfilter
im_roi1 = roi1.interactive(im, color="red")
im_roi1_mean = hs.interactive(im_roi1.mean,
                        event=roi1.events.changed,
                        recompute_out_event=None)

cl_img = im_roi1_mean.data

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(cl_img, cmap='viridis')


plt.savefig('cl_img.png', dpi=600, transparent=True)



## Plot Numpy Fits from HPC

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter
from scipy.linalg import lstsq

# ---- Change this to your folder ----
data_dir = r"C:\Users\cobia\OneDrive - University of Cambridge\CL\numpy_fits"

# ---- Helper functions ----

def remove_outliers(arr, z_thresh=5.0, filter_size=3):
    """
    Remove extreme outliers based on a simple z-score-like measure.
    Replace outliers with local median.
    """
    # Median filter to estimate local background
    med = median_filter(arr, size=filter_size)
    diff = arr - med
    mad = np.median(np.abs(diff))
    outliers = np.abs(diff) > z_thresh * mad
    arr_clean = arr.copy()
    arr_clean[outliers] = med[outliers]  # interpolate with local median
    return arr_clean

def flatten_plane(arr):
    """
    Fit a plane z = ax + by + c to the data and subtract it.
    """
    nrows, ncols = arr.shape
    X, Y = np.meshgrid(np.arange(ncols), np.arange(nrows))
    A = np.c_[X.ravel(), Y.ravel(), np.ones(nrows*ncols)]
    z = arr.ravel()
    coeff, _, _, _ = lstsq(A, z)
    plane = (coeff[0]*X + coeff[1]*Y + coeff[2])
    return arr - plane

# ---- Main loop ----

plt.close('all')  # Close all previous plots

intensity_files = glob.glob(os.path.join(data_dir, "*_m_intensity.npy"))

for inten_file in intensity_files:
    prefix = inten_file.replace("_m_intensity.npy", "")
    centre_file = prefix + "_m_centre.npy"

    if not os.path.exists(centre_file):
        print(f"⚠️ Skipping {inten_file}, no matching centre file found.")
        continue

    # Load arrays
    inten = np.load(inten_file)
    centre = np.load(centre_file)

    # Remove outliers
    inten_clean = remove_outliers(inten)
    centre_clean = remove_outliers(centre)

    # Flatten centre by subtracting mean plane
    centre_flat = flatten_plane(centre_clean)

    # Plot side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    im0 = axes[0].imshow(inten_clean, cmap="viridis")
    axes[0].set_title(os.path.basename(inten_file))
    plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(centre_flat, cmap="gray")
    axes[1].set_title(os.path.basename(centre_file) + " (flattened)")
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()
